In [4]:
import pandas as pd

# 파일 읽기 (한글 인코딩 처리)
df = pd.read_csv(r"C:\Users\user\Desktop\파프\ESI_PSYCO.csv", index_col=0, encoding='cp949')

path = r"C:\Users\user\Desktop\파프\ESI_PSYCO.csv"

# 인덱스를 문자열로 변환한 후 "20" 제거
df.index = df.index.astype(str).str.replace("^20", "", regex=True)

# 결과 확인
print(df.head())

# 다시 같은 위치에 저장 (인덱스 포함, 인코딩 유지)
df.to_csv(path, encoding='cp949')

       value   변동폭      로그변환   BoxCox변환
03.01  101.6   NaN       NaN        NaN
03.02  102.9   1.3  3.165475  64.936098
03.03   92.6 -10.3  2.493205  24.296948
03.04   98.4   5.8  3.339322  83.549226
03.05   96.9  -1.5  3.039749  54.093896


In [26]:

esi_path=r"C:\Users\user\Desktop\파프\ESI_PSYCO.csv"
# 1. ESI 불러오기
esi = pd.read_csv(esi_path, encoding='cp949', index_col=0)
esi.index.name = 'date'
esi.reset_index(inplace=True)
type(esi.index[0])

int

In [196]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests

def granger_lag_summary(issue_path, esi_path=r"C:\Users\user\Desktop\파프\ESI_PSYCO.csv"):
    # 1. ESI 불러오기
    esi = pd.read_csv(esi_path, encoding='cp949')

    # 2. 이슈 기사 수 불러오기
    issue = pd.read_excel(issue_path)
    news_col = [col for col in issue.columns if col != 'date'][0]
    issue = issue.rename(columns={news_col: 'news_count'})


    # 4. 병합
    df = pd.merge(esi[['date', 'BoxCox변환']], issue[['date', 'news_count']], on='date', how='inner').dropna()

    # ✅ 병합 결과 출력
    print(f"\n📌 병합된 데이터 ({len(df)} rows):")
    display(df.head(20))

    if len(df) < 5:
        print("❗ 데이터가 5행 미만이라 인과성 검정 생략됨.")
        return

    # 5. 인과성 검정 요약
    results_summary = []
    '''
    # lag = -2
    df_lag_m2 = df.copy()
    df_lag_m2['news_count_shifted'] = df_lag_m2['news_count'].shift(-2)
    df_lag_m2 = df_lag_m2.dropna()
    try:
        p = grangercausalitytests(df_lag_m2[['BoxCox변환', 'news_count_shifted']], maxlag=1, verbose=False)[1][0]['ssr_ftest'][1]
    except:
        p = None
    results_summary.append({'lag': -2, 'p-value': p})
    '''

    # lag = -1
    df_lag_m1 = df.copy()
    df_lag_m1['news_count_shifted'] = df_lag_m1['news_count'].shift(-1)
    df_lag_m1 = df_lag_m1.dropna()
    try:
        p = grangercausalitytests(df_lag_m1[['BoxCox변환', 'news_count_shifted']], maxlag=1, verbose=False)[1][0]['ssr_ftest'][1]
    except:
        p = None
    results_summary.append({'lag': -1, 'p-value': p})

    # lag = 0
    df_lag0 = df.copy()
    df_lag0['BoxCox변환_shifted'] = df_lag0['BoxCox변환'].shift(-1)
    df_lag0 = df_lag0.dropna()
    try:
        p = grangercausalitytests(df_lag0[['BoxCox변환_shifted', 'news_count']], maxlag=1, verbose=False)[1][0]['ssr_ftest'][1]
    except:
        p = None
    results_summary.append({'lag': 0, 'p-value': p})

    # lag = 1~3
    for lag in [1, 2, 3]:
        try:
            result = grangercausalitytests(df[['BoxCox변환', 'news_count']], maxlag=lag, verbose=False)
            p = result[lag][0]['ssr_ftest'][1]
        except:
            p = None
        results_summary.append({'lag': lag, 'p-value': p})

    # ✅ 요약 출력
    print("\n📈 인과성 검정 요약 (F-test 기준):")
    for row in results_summary:
        pval = row['p-value']
        if pval is None:
            print(f"lag = {row['lag']:>2} | p-value = N/A     | 🚫 계산 실패")
        else:
            status = "✅ 유의함" if pval < 0.05 else "❌ 유의하지 않음"
            print(f"lag = {row['lag']:>2} | p-value = {pval:.4f} | {status}")


In [198]:
import os

# 대상 폴더 경로
folder_path = r"C:\Users\user\Desktop\파프\이슈"

# 폴더 내 모든 파일의 전체 경로 리스트
file_paths = [os.path.join(folder_path, fname) for fname in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, fname))]


for path in file_paths:
    print(path)
    granger_lag_summary(path)
    print(path)
    print('/n')
    print('/n')



C:\Users\user\Desktop\파프\이슈\21대총선.xlsx

📌 병합된 데이터 (24 rows):


,date,BoxCox변환,news_count
0,20.01,70.574928,5632
1,20.02,31.393054,5008
2,20.03,0.000000,7670
3,20.04,33.656800,15514
4,20.05,68.544885,5028
5,20.06,78.870311,2395
6,20.07,95.817557,1428
7,20.08,85.702654,1106
8,20.09,39.002867,1049
9,20.10,108.579344,1075



📈 인과성 검정 요약 (F-test 기준):
lag = -1 | p-value = 0.0286 | ✅ 유의함
lag =  0 | p-value = 0.8387 | ❌ 유의하지 않음
lag =  1 | p-value = 0.5922 | ❌ 유의하지 않음
lag =  2 | p-value = 0.8266 | ❌ 유의하지 않음
lag =  3 | p-value = 0.4597 | ❌ 유의하지 않음
C:\Users\user\Desktop\파프\이슈\21대총선.xlsx
/n
/n
C:\Users\user\Desktop\파프\이슈\G8.xlsx

📌 병합된 데이터 (36 rows):


,date,BoxCox변환,news_count
0,9.01,87.872508,286
1,9.02,70.167496,303
2,9.03,70.983069,897
3,9.04,138.980288,1571
4,9.05,80.987022,738
5,9.06,72.211726,974
6,9.07,64.142152,1318
7,9.08,95.370756,365
8,9.09,66.132502,2185
9,9.10,70.574928,1751



📈 인과성 검정 요약 (F-test 기준):
lag = -1 | p-value = 0.7761 | ❌ 유의하지 않음
lag =  0 | p-value = 0.4091 | ❌ 유의하지 않음
lag =  1 | p-value = 0.5572 | ❌ 유의하지 않음
lag =  2 | p-value = 0.7731 | ❌ 유의하지 않음
lag =  3 | p-value = 0.9342 | ❌ 유의하지 않음
C:\Users\user\Desktop\파프\이슈\G8.xlsx
/n
/n
C:\Users\user\Desktop\파프\이슈\imf.xlsx

📌 병합된 데이터 (39 rows):


...


📈 인과성 검정 요약 (F-test 기준):
lag = -1 | p-value = 0.8969 | ❌ 유의하지 않음
lag =  0 | p-value = 0.8848 | ❌ 유의하지 않음
lag =  1 | p-value = 0.2033 | ❌ 유의하지 않음
lag =  2 | p-value = 0.4165 | ❌ 유의하지 않음
lag =  3 | p-value = N/A     | 🚫 계산 실패
C:\Users\user\Desktop\파프\이슈\조기대선.xlsx
/n
/n
C:\Users\user\Desktop\파프\이슈\조류.xlsx

📌 병합된 데이터 (37 rows):


,date,BoxCox변환,news_count
0,3.02,64.936098,198
1,3.03,24.296948,182
2,3.04,83.549226,212
3,3.05,54.093896,240
4,3.06,47.446549,227
5,3.07,58.282313,268
6,3.08,84.839303,261
7,3.09,61.778019,249
8,3.10,58.667698,287
9,3.11,76.352665,279



📈 인과성 검정 요약 (F-test 기준):
lag = -1 | p-value = 0.6338 | ❌ 유의하지 않음
lag =  0 | p-value = 0.8796 | ❌ 유의하지 않음
lag =  1 | p-value = 0.3765 | ❌ 유의하지 않음
lag =  2 | p-value = 0.7312 | ❌ 유의하지 않음
lag =  3 | p-value = 0.2929 | ❌ 유의하지 않음
C:\Users\user\Desktop\파프\이슈\조류.xlsx
/n
/n
C:\Users\user\Desktop\파프\이슈\코로나.xlsx

📌 병합된 데이터 (72 rows):


,date,BoxCox변환,news_count
0,19.01,49.266757,20
1,19.02,82.265120,9
2,19.03,56.366879,35
3,19.04,64.936098,48
4,19.05,46.364324,25
5,19.06,62.170186,45
6,19.07,49.266757,34
7,19.08,56.748428,16
8,19.09,71.801469,72
9,19.10,55.606104,38



📈 인과성 검정 요약 (F-test 기준):
lag = -1 | p-value = 0.4967 | ❌ 유의하지 않음
lag =  0 | p-value = 0.0091 | ✅ 유의함
lag =  1 | p-value = 0.3558 | ❌ 유의하지 않음
lag =  2 | p-value = 0.0000 | ✅ 유의함
lag =  3 | p-value = 0.0000 | ✅ 유의함
C:\Users\user\Desktop\파프\이슈\코로나.xlsx
/n
/n


In [221]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests
import warnings
warnings.filterwarnings("ignore")

def granger_lag_correlation_summary(issue_path, esi_path=r"C:\Users\user\Desktop\파프\ESI_PSYCO.csv"):
    issue_name = issue_path.split('\\')[-1].replace('.xlsx', '')

    # 1. 데이터 로드
    esi = pd.read_csv(esi_path, encoding='cp949')
    issue = pd.read_excel(issue_path)
    news_col = [col for col in issue.columns if col != 'date'][0]
    issue = issue.rename(columns={news_col: 'news_count'})
    df = pd.merge(esi[['date', 'BoxCox변환']], issue[['date', 'news_count']], on='date', how='inner').dropna()

    if len(df) < 5:
        return pd.DataFrame([{
            '이슈': issue_name,
            '최적 lag': None,
            'Granger p-value': None,
            '상관계수': None,
            '영향 방향': '데이터 부족'
        }])

    # 2. Granger 인과성 검정
    def try_granger(x, y):
        try:
            test = grangercausalitytests(pd.DataFrame({0: x, 1: y}), maxlag=1, verbose=False)
            return test[1][0]['ssr_ftest'][1]
        except:
            return None

    results = []
    for lag in [-2,-1, 0, 1, 2, 3]:
        df_lag = df.copy()
        df_lag['shifted'] = df_lag['news_count'].shift(-lag)
        df_lag = df_lag.dropna()
        p = try_granger(df_lag['BoxCox변환'], df_lag['shifted'])
        if p is not None:
            results.append({'lag': lag, 'pval': p})

    # 3. 가장 작은 p-value 가진 lag 선택
    valid = [r for r in results if r['pval'] < 0.05]
    if not valid:
        return pd.DataFrame([{
            '이슈': issue_name,
            '최적 lag': None,
            'Granger p-value': None,
            '상관계수': None,
            '영향 방향': '유의한 lag 없음'
        }])

    best = min(valid, key=lambda r: r['pval'])
    best_lag = best['lag']
    best_p = best['pval']

    # 4. 선택된 lag 기준으로 상관계수 계산
    df_best = df.copy()
    df_best['shifted'] = df_best['news_count'].shift(-best_lag)
    df_best = df_best.dropna()
    df_best = df_best[df_best['shifted'] > 60]

    if len(df_best) < 5:
        return pd.DataFrame([{
            '이슈': issue_name,
            '최적 lag': best_lag,
            'Granger p-value': best_p,
            '상관계수': None,
            '영향 방향': 'x 유효구간 부족'
        }])

    corr = df_best['shifted'].corr(df_best['BoxCox변환'])
    direction = '양(+) 영향' if corr > 0 else '음(-) 영향'

    return pd.DataFrame([{
        '이슈': issue_name,
        '최적 lag': best_lag,
        'Granger p-value': best_p,
        '상관계수': corr,
        '영향 방향': direction
    }])


In [222]:
import os
folder = r"C:\Users\user\Desktop\파프\이슈"
files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.xlsx')]

summary_df = pd.concat([granger_lag_correlation_summary(path) for path in files], ignore_index=True)
display(summary_df)


,이슈,최적 lag,Granger p-value,상관계수,영향 방향
0,21대총선,2,0.001298,-0.38245,음(-) 영향
1,G8,None,NaN,None,유의한 lag 없음
2,imf,-2,0.016543,0.204315,양(+) 영향
3,sars,None,NaN,None,유의한 lag 없음
4,계엄,1,0.011623,-0.049175,음(-) 영향
5,공무원노조,-2,0.006166,0.322266,양(+) 영향
6,관세,3,0.003551,0.026414,양(+) 영향
7,광우병,None,NaN,None,유의한 lag 없음
8,김대중 서거,None,NaN,None,유의한 lag 없음
9,노무현,-1,0.016247,-0.127478,음(-) 영향


In [145]:
from statsmodels.tsa.tsatools import lagmat
import statsmodels.api as sm

def get_granger_model_info(df, y_col, x_col, lag):
    """지정된 lag에 대해 회귀계수 및 R² 반환"""
    df_clean = df[[y_col, x_col]].dropna().reset_index(drop=True)
    data = lagmat(df_clean, maxlag=lag, trim='both', original='in')

    y = data[:, 0]  # 현재 시점의 y
    x = data[:, 2:(2 + lag)]  # x의 lag 1~lag n
    x = sm.add_constant(x)

    model = sm.OLS(y, x).fit()
    return model.params, model.rsquared


In [152]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.tsatools import lagmat
import statsmodels.api as sm

def get_granger_model_info(df, y_col, x_col, lag):
    """지정된 lag에 대해 회귀계수 및 R² 반환"""
    df_clean = df[[y_col, x_col]].dropna().reset_index(drop=True)
    data = lagmat(df_clean, maxlag=lag, trim='both', original='in')

    y = data[:, 0]  # 현재 시점의 y
    x = data[:, 2:(2 + lag)]  # x의 lag 1~lag n
    x = sm.add_constant(x)

    model = sm.OLS(y, x).fit()
    return model.params, model.rsquared

def granger_lag_summary(issue_path, esi_path=r"C:\Users\user\Desktop\파프\ESI_PSYCO.csv"):
    # 1. ESI 불러오기
    esi = pd.read_csv(esi_path, encoding='cp949')

    # 2. 이슈 기사 수 불러오기
    issue = pd.read_excel(issue_path)
    news_col = [col for col in issue.columns if col != 'date'][0]
    issue = issue.rename(columns={news_col: 'news_count'})

    # 3. 병합
    df = pd.merge(esi[['date', 'value']], issue[['date', 'news_count']], on='date', how='inner').dropna()

    print(f"\n📌 병합된 데이터 ({len(df)} rows):")
    display(df.head())

    if len(df) < 5:
        print("❗ 데이터가 5행 미만이라 인과성 검정 생략됨.")
        return

    # 4. Granger 인과성 검정
    results_summary = []

    # lag = -2, -1, 0
    for label, shift in zip([-2, -1], [-2, -1]):
        df_shifted = df.copy()
        df_shifted['news_count_shifted'] = df_shifted['news_count'].shift(shift)
        df_shifted = df_shifted.dropna()
        try:
            pval = grangercausalitytests(df_shifted[['BoxCox변환', 'news_count_shifted']], maxlag=1, verbose=False)[1][0]['ssr_ftest'][1]
        except:
            pval = None
        results_summary.append({'lag': label, 'p-value': pval})

    # lag = 0 (Y를 한 칸 뒤로 밀기)
    df_lag0 = df.copy()
    df_lag0['BoxCox변환_shifted'] = df_lag0['BoxCox변환'].shift(-1)
    df_lag0 = df_lag0.dropna()
    try:
        pval = grangercausalitytests(df_lag0[['BoxCox변환_shifted', 'news_count']], maxlag=1, verbose=False)[1][0]['ssr_ftest'][1]
    except:
        pval = None
    results_summary.append({'lag': 0, 'p-value': pval})

    # lag = 1 ~ 3
    for lag in [1, 2, 3]:
        try:
            result = grangercausalitytests(df[['BoxCox변환', 'news_count']], maxlag=lag, verbose=False)
            pval = result[lag][0]['ssr_ftest'][1]
        except:
            pval = None
        results_summary.append({'lag': lag, 'p-value': pval})

    # 5. 출력
    print("\n📈 인과성 검정 요약 (F-test 기준):")
    for row in results_summary:
        pval = row['p-value']
        if pval is None:
            print(f"lag = {row['lag']:>2} | p-value = N/A     | 🚫 계산 실패")
        else:
            status = "✅ 유의함" if pval < 0.05 else "❌ 유의하지 않음"
            print(f"lag = {row['lag']:>2} | p-value = {pval:.4f} | {status}")

    # 6. 유의한 lag에 대해 회귀 계수 추출
        # 6. 가장 유의한 lag에 대해 회귀 계수 추출
    sig_lags = [row for row in results_summary if row['p-value'] is not None and row['p-value'] < 0.05 and row['lag'] > 0]

    if not sig_lags:
        print("\n❌ 유의한 lag가 없어 회귀 계수 분석 생략됨.")
        return

    # 가장 작은 p-value 가진 lag 선택
    best_row = min(sig_lags, key=lambda r: r['p-value'])
    best_lag = best_row['lag']
    best_p = best_row['p-value']

    print(f"\n🌟 가장 유의한 lag = {best_lag} (p-value = {best_p:.4f})")
    coef, r2 = get_granger_model_info(df, y_col='BoxCox변환', x_col='news_count', lag=best_lag)

    # 회귀 계수 출력 (소수점 2자리)
    print("📌 회귀 계수 (소수점 2자리):")
    if hasattr(coef, "items"):
        for name, val in coef.items():
            print(f"  {name:>15}: {val:.2f}")
    else:
        # numpy 배열인 경우 인덱스를 붙여 출력
        for i, val in enumerate(coef):
            print(f"  coef[{i}]: {val:.2f}")



In [153]:
granger_lag_summary(r"C:\Users\user\Desktop\파프\이슈\코로나.xlsx")



📌 병합된 데이터 (72 rows):


,date,BoxCox변환,news_count
0,19.01,49.266757,20
1,19.02,82.265120,9
2,19.03,56.366879,35
3,19.04,64.936098,48
4,19.05,46.364324,25



📈 인과성 검정 요약 (F-test 기준):
lag = -2 | p-value = 0.9351 | ❌ 유의하지 않음
lag = -1 | p-value = 0.4967 | ❌ 유의하지 않음
lag =  0 | p-value = 0.0091 | ✅ 유의함
lag =  1 | p-value = 0.3558 | ❌ 유의하지 않음
lag =  2 | p-value = 0.0000 | ✅ 유의함
lag =  3 | p-value = 0.0000 | ✅ 유의함

🌟 가장 유의한 lag = 3 (p-value = 0.0000)
📌 회귀 계수 (소수점 2자리):
  coef[0]: 45.14
  coef[1]: 0.20
  coef[2]: 0.00
  coef[3]: 0.00


In [28]:
import os
import pandas as pd

def normalize_date(val):
    val = str(val).strip()
    
    # case 1: 6자리 정수 형태 (예: 200708)
    if val.isdigit() and len(val) == 6:
        year = val[2:4]
        month = val[4:6]
        return f"{year.zfill(2)}.{month.zfill(2)}"
    
    # case 2: 소수점 구분 (예: 4.6, 10.12)
    if '.' in val:
        parts = val.split('.')
        year = parts[0].zfill(2)
        month = parts[1].zfill(2)
        return f"{year}.{month}"
    
    return val  # fallback (예외 입력)

def convert_and_save(filepath, save_dir=r"C:\Users\user\Desktop\파프\이슈"):
    # 1. 파일 불러오기
    df = pd.read_excel(filepath)

    # 2. date 열 존재 시 변환
    if 'date' not in df.columns:
        raise ValueError("❗ 'date' 열이 존재하지 않습니다.")
    
    df['date'] = df['date'].astype(str).apply(normalize_date)

    # 3. 저장 경로 설정
    filename = os.path.basename(filepath)  # 파일명만 추출
    save_path = os.path.join(save_dir, filename)

    # 4. 저장 폴더가 없다면 생성
    os.makedirs(save_dir, exist_ok=True)

    # 5. 저장
    df.to_excel(save_path, index=False)
    print(f"✅ 변환 완료: {save_path}")


# 대상 폴더 경로
folder_path = r"C:\Users\user\Desktop\파프\이슈 주별 계산수_앞에 1년 덧붙인거"

# 폴더 내 모든 파일의 전체 경로 리스트
file_paths = [os.path.join(folder_path, fname) for fname in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, fname))]


for path in file_paths:
    print(path)
    convert_and_save(path)


C:\Users\user\Desktop\파프\이슈 주별 계산수_앞에 1년 덧붙인거\21대총선.xlsx
✅ 변환 완료: C:\Users\user\Desktop\파프\이슈\21대총선.xlsx
C:\Users\user\Desktop\파프\이슈 주별 계산수_앞에 1년 덧붙인거\G8.xlsx
✅ 변환 완료: C:\Users\user\Desktop\파프\이슈\G8.xlsx
C:\Users\user\Desktop\파프\이슈 주별 계산수_앞에 1년 덧붙인거\imf.xlsx
✅ 변환 완료: C:\Users\user\Desktop\파프\이슈\imf.xlsx
C:\Users\user\Desktop\파프\이슈 주별 계산수_앞에 1년 덧붙인거\sars.xlsx
✅ 변환 완료: C:\Users\user\Desktop\파프\이슈\sars.xlsx
C:\Users\user\Desktop\파프\이슈 주별 계산수_앞에 1년 덧붙인거\계엄.xlsx
✅ 변환 완료: C:\Users\user\Desktop\파프\이슈\계엄.xlsx
C:\Users\user\Desktop\파프\이슈 주별 계산수_앞에 1년 덧붙인거\공무원노조.xlsx
✅ 변환 완료: C:\Users\user\Desktop\파프\이슈\공무원노조.xlsx
C:\Users\user\Desktop\파프\이슈 주별 계산수_앞에 1년 덧붙인거\관세.xlsx
✅ 변환 완료: C:\Users\user\Desktop\파프\이슈\관세.xlsx
C:\Users\user\Desktop\파프\이슈 주별 계산수_앞에 1년 덧붙인거\광우병.xlsx
✅ 변환 완료: C:\Users\user\Desktop\파프\이슈\광우병.xlsx
C:\Users\user\Desktop\파프\이슈 주별 계산수_앞에 1년 덧붙인거\김대중 서거.xlsx
✅ 변환 완료: C:\Users\user\Desktop\파프\이슈\김대중 서거.xlsx
C:\Users\user\Desktop\파프\이슈 주별 계산수_앞에 1년 덧붙인거\노무현.xlsx
✅ 변환 완료: C:\Users\user\Des

In [173]:
import os
import pandas as pd

# 사건 파일 경로
issue_dir = r"C:\Users\user\Desktop\파프\이슈"
issue_files = [f for f in os.listdir(issue_dir) if f.endswith('.xlsx')]

all_dates = set()
dfs = []

# 1. 각 파일을 처리하여 date, 사건명(count) 열 만들기
for file in issue_files:
    issue_name = os.path.splitext(file)[0]  # 파일명 (ex: 코로나)
    df = pd.read_excel(os.path.join(issue_dir, file))
    df.columns = ['date', issue_name]  # 열 이름 통일
    df['date'] = df['date'].astype(float)
    all_dates.update(df['date'].unique())
    dfs.append(df)

# 2. 전체 날짜 범위로 DataFrame 구성
all_dates = sorted(list(all_dates))
df_all = pd.DataFrame({'date': all_dates})

# 3. 사건별 데이터 merge
for df in dfs:
    df_all = df_all.merge(df, on='date', how='left')

# 4. 결측값을 0으로 채움
df_all.fillna(0, inplace=True)
df_all = df_all.sort_values('date') 
# 5. 결과 저장 (선택)
df_all.to_csv(r"C:\Users\user\Desktop\파프\이슈_통합.csv", index=False, encoding='utf-8-sig')

print("완료: 사건별 기사 수 통합 테이블이 생성되었습니다.")


완료: 사건별 기사 수 통합 테이블이 생성되었습니다.


In [174]:
import pandas as pd
import numpy as np
from itertools import combinations, product
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.linear_model import LinearRegression

# 날짜 파싱 함수
def parse_yy_mm(s):
    try:
        yy, mm = s.split('.')
        yy = yy.zfill(2)
        mm = mm.zfill(2)
        return pd.to_datetime(f"20{yy}-{mm}-01")
    except:
        return pd.NaT

# 데이터 불러오기

issue_df = pd.read_csv(
    r"C:\Users\user\Desktop\파프\이슈_통합.csv", 
    dtype={"date": str},
    encoding = 'utf-8-sig'
)


esi_df = pd.read_csv(r"C:\Users\user\Desktop\파프\ESI_PSYCO.csv", dtype={"date": str},  encoding = 'cp949')

# 2. 데이터 전처리 -------------------------------------------------
# 날짜 형식 통일 (예: '03.1' -> '03.01')
def format_date(s):
    parts = s.strip().split('.')
    if len(parts) == 2:
        yy = parts[0].zfill(2)  # '3' → '03'
        mm = parts[1].zfill(2)  # '6' → '06'
        return f"{yy}.{mm}"
    return s  # 예외 처리
issue_df['date'] = issue_df['date'].astype(str).apply(format_date)
esi_df['date'] = esi_df['date'].astype(str).apply(format_date)


# 데이터 병합
merged = pd.merge(
    issue_df, 
    esi_df[['date', 'value']], 
    on='date', 
    how='inner'
).sort_values('date').reset_index(drop=True)

# 3. Lag 변수 생성 -------------------------------------------------
event_cols = [col for col in merged.columns if col not in ['date', 'value']]

for lag in range(1, 4):
    for col in event_cols:
        merged[f'{col}_lag{lag}'] = merged[col].shift(lag)
        
print("분석 실행")

# 4. 분석 실행 -----------------------------------------------------
# 상단에 tqdm 임포트 추가
from tqdm import tqdm

results = []

for idx in tqdm(range(3, len(merged)), desc="월별 분석 진행 중"):
    current_data = merged.iloc[idx]
    month = current_data['date']
    
    print(f"\n🔍 [{idx}] 월: {month} 분석 시작")

    candidate_vars = []
    for col in event_cols:
        for lag in range(1, 4):
            lag_col = f'{col}_lag{lag}'
            val = current_data[lag_col]
            if not pd.isna(val) and val != 0:
                candidate_vars.append((col, lag, val))
    
    best_combo, best_r2, best_pvals = None, -np.inf, []
    
    if candidate_vars:
        print(f"  ▶️ 후보 변수 개수: {len(candidate_vars)}")

        for r in tqdm(range(1, min(4, len(candidate_vars)+1)), 
                      desc=f"  ↪ 조합 size={r}", leave=False):
            for combo in combinations(candidate_vars, r):
                X_cols = [f'{col}_lag{lag}' for col, lag, _ in combo]
                try:
                    X = merged[X_cols].iloc[:idx].dropna()
                    y = merged['value'].iloc[1:idx+1][X.index]
                    
                    if len(X) < 5:
                        print(f"    ⚠️ 샘플 수 부족 (len={len(X)}): {X_cols}")
                        continue
                    
                    model = LinearRegression().fit(X, y)
                    r2 = model.score(X, y)

                    pvals = []
                    for cname in X_cols:
                        try:
                            test_result = grangercausalitytests(
                                merged[[cname, 'value']].dropna(), 
                                maxlag=1, 
                                verbose=False
                            )
                            pval = test_result[1][0]['ssr_ftest'][1]
                            pvals.append(pval)
                        except Exception as e:
                            print(f"    ❌ Granger 실패 - {cname}: {e}")
                            pvals.append(np.nan)

                    if r2 > best_r2 and not np.isnan(r2):
                        best_r2 = r2
                        best_combo = combo
                        best_pvals = pvals
                        print(f"    ✅ 새로운 최적 조합 (r²={r2:.4f}): {X_cols}")
                        
                except Exception as e:
                    print(f"    ❌ 회귀 실패 - {X_cols}: {e}")
                    continue
    else:
        print("  ⚠️ 유의한 후보 없음")

    results.append({
        'date': month,
        'best_combo': best_combo,
        'r2': best_r2,
        'pvals': best_pvals
    })

print("\n🎉 전체 분석 완료!")


분석 실행


월별 분석 진행 중:   0%|                           | 0/226 [00:00<?, ?it/s]


🔍 [3] 월: 03.03 분석 시작
  ▶️ 후보 변수 개수: 15



  ↪ 조합 size=3:   0%|                               | 0/3 [00:00<?, ?it/s]

    ⚠️ 샘플 수 부족 (len=2): ['sars_lag1']
    ⚠️ 샘플 수 부족 (len=1): ['sars_lag2']
    ⚠️ 샘플 수 부족 (len=0): ['sars_lag3']
    ⚠️ 샘플 수 부족 (len=2): ['노무현_lag1']
    ⚠️ 샘플 수 부족 (len=1): ['노무현_lag2']
    ⚠️ 샘플 수 부족 (len=0): ['노무현_lag3']
    ⚠️ 샘플 수 부족 (len=2): ['미국대선_lag1']
    ⚠️ 샘플 수 부족 (len=1): ['미국대선_lag2']
    ⚠️ 샘플 수 부족 (len=0): ['미국대선_lag3']
    ⚠️ 샘플 수 부족 (len=2): ['이라크,파병_lag1']
    ⚠️ 샘플 수 부족 (len=1): ['이라크,파병_lag2']
    ⚠️ 샘플 수 부족 (len=0): ['이라크,파병_lag3']
    ⚠️ 샘플 수 부족 (len=2): ['조류_lag1']
    ⚠️ 샘플 수 부족 (len=1): ['조류_lag2']
    ⚠️ 샘플 수 부족 (len=0): ['조류_lag3']
    ⚠️ 샘플 수 부족 (len=1): ['sars_lag1', 'sars_lag2']
    ⚠️ 샘플 수 부족 (len=0): ['sars_lag1', 'sars_lag3']
    ⚠️ 샘플 수 부족 (len=2): ['sars_lag1', '노무현_lag1']
    ⚠️ 샘플 수 부족 (len=1): ['sars_lag1', '노무현_lag2']
    ⚠️ 샘플 수 부족 (len=0): ['sars_lag1', '노무현_lag3']
    ⚠️ 샘플 수 부족 (len=2): ['sars_lag1', '미국대선_lag1']
    ⚠️ 샘플 수 부족 (len=1): ['sars_lag1', '미국대선_lag2']
    ⚠️ 샘플 수 부족 (len=0): ['sars_lag1', '미국대선_lag3']
    ⚠️ 샘플 수 부족 (len=2): ['sa

...

    ✅ 새로운 최적 조합 (r²=0.4522): ['sars_lag1', '노무현_lag1']
    ✅ 새로운 최적 조합 (r²=0.5588): ['sars_lag2', '노무현_lag1']
    ✅ 새로운 최적 조합 (r²=0.7565): ['sars_lag2', '조류_lag1']
    ✅ 새로운 최적 조합 (r²=0.8453): ['이라크,파병_lag1', '조류_lag1']



  ↪ 조합 size=3:  67%|███████████████▎       | 2/3 [00:03<00:01,  1.75s/it]

    ✅ 새로운 최적 조합 (r²=0.8594): ['sars_lag1', '이라크,파병_lag1', '조류_lag1']
    ✅ 새로운 최적 조합 (r²=0.8704): ['sars_lag1', '이라크,파병_lag3', '조류_lag2']
    ✅ 새로운 최적 조합 (r²=0.9778): ['sars_lag2', 'sars_lag3', '미국대선_lag2']



월별 분석 진행 중:   3%|▌                  | 6/226 [00:45<27:50,  7.59s/it]       


KeyboardInterrupt: 

In [ ]:
from datetime import datetime

# 날짜 정렬 함수
def parse_date_string(date_str):
    try:
        yy, mm = date_str.strip().split('.')
        yy = int(yy)
        if yy < 100:
            yy += 2000 if yy < 50 else 1900
        return datetime(yy, int(mm), 1)
    except:
        return datetime.max

# R² 해석 함수
def interpret_r2(r2):
    if r2 >= 0.5:
        return "✅ 유의미한 설명력"
    elif r2 >= 0.3:
        return "⚠️ 보통 수준의 설명력"
    else:
        return "❌ 설명력 낮음 (비유의미)"

# 결과 정렬 및 출력
results_sorted = sorted(results, key=lambda r: parse_date_string(r['date']))

print("\n\n최종 분석 결과:")
for res in results_sorted:
    print(f"\n📅 월: {res['date']}")
    if not res['best_combo']:
        print("  → 유의한 사건 조합 없음")
    else:
        print("  → 최적 사건 조합:")
        for (col, lag, val), pval in zip(res['best_combo'], res['pvals']):
            print(f"    • {col} (lag {lag}, 기사수 {val}) - Granger p-value: {pval:.4f}")
        print(f"  → 모델 설명력(R²): {res['r2']:.4f} → {interpret_r2(res['r2'])}")
    print("-" * 60)


In [114]:
from collections import defaultdict
import pandas as pd

# 날짜별 정보를 담을 딕셔너리
summary_by_date = defaultdict(dict)

for res in results_sorted:
    date = res['date']
    summary_by_date['r2'][date] = res['r2']
    summary_by_date['r2_interpretation'][date] = interpret_r2(res['r2'])

    if res['best_combo']:
        for i, ((col, lag, val), pval) in enumerate(zip(res['best_combo'], res['pvals']), 1):
            summary_by_date[f'event_{i}'][date] = f"{col}_lag{lag}={val}"
            summary_by_date[f'pval_{i}'][date] = f"{pval:.4f}"
    else:
        summary_by_date['event_1'][date] = '없음'
        summary_by_date['pval_1'][date] = 'NaN'

# DataFrame으로 변환하고 날짜를 열로
wide_df = pd.DataFrame(summary_by_date)

# 전치해서 날짜가 열이 되게 함
wide_df = wide_df.T

# 저장
wide_df.to_csv(r"C:\Users\user\Desktop\파프\분석결과_날짜열버전.csv", encoding='utf-8-sig')

print("\n✅ CSV 저장 완료: 분석결과_날짜열버전.csv (날짜가 열로)")



✅ CSV 저장 완료: 분석결과_날짜열버전.csv (날짜가 열로)


In [115]:
# 엑셀로 저장
excel_path = r"C:\Users\user\Desktop\파프\분석결과_날짜열버전.xlsx"
wide_df.to_excel(excel_path, sheet_name='분석결과', encoding='utf-8-sig')

print(f"\n✅ 엑셀 저장 완료: {excel_path}")



✅ 엑셀 저장 완료: C:\Users\user\Desktop\파프\분석결과_날짜열버전.xlsx


In [94]:
import os
import pandas as pd

# 사건 파일 경로
issue_dir = r"C:\Users\user\Desktop\파프\이슈"
issue_files = [f for f in os.listdir(issue_dir) if f.endswith('.xlsx')]

all_dates = set()
dfs = []

# 1. 각 파일을 처리하여 'date', 사건명 열 구성
for file in issue_files:
    issue_name = os.path.splitext(file)[0]  # 파일명 (확장자 제거)
    file_path = os.path.join(issue_dir, file)
    
    try:
        df = pd.read_excel(file_path)
        if df.shape[1] != 2:
            print(f"⚠️ 열 개수 문제: {file}")
            continue

        df.columns = ['date', issue_name]  # 열 이름 통일
        df['date'] = df['date'].astype(str).str.strip()
        df = df.groupby('date').sum().reset_index()  # 중복 date 합산

        all_dates.update(df['date'].unique())
        dfs.append(df)
    except Exception as e:
        print(f"❌ {file} 읽기 실패: {e}")

# 2. 전체 날짜 목록을 기준으로 초기 DataFrame 생성
all_dates = sorted(all_dates)
df_all = pd.DataFrame({'date': all_dates})

# 3. 사건별 데이터 병합
for df in dfs:
    df_all = df_all.merge(df, on='date', how='left')

# 4. 결측값 0으로 대체
df_all.fillna(0, inplace=True)

# 5. 날짜 정렬
df_all = df_all.sort_values('date').reset_index(drop=True)

# 6. 저장
output_path = r"C:\Users\user\Desktop\파프\이슈_통합.csv"
df_all.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ 완료: {output_path} 에 통합 테이블이 저장되었습니다.")


✅ 완료: C:\Users\user\Desktop\파프\이슈_통합.csv 에 통합 테이블이 저장되었습니다.
